In [470]:
class numlink:

    def __init__(self, initial_state):
        self.node = node(initial_state,tuple([1,2]),None)
        self.start_finish_dict={}
        self.closed_list=[]
        self.sf_list=[]

#     Will get start and finish states for each number in the puzzle
    def declare_SF(self):
        for i in range(len(self.node.state)):
            for j in range(len(self.node.state[i])):
                if self.node.state[i][j] !='':
                    if self.node.state[i][j] in self.start_finish_dict:
                        self.start_finish_dict[self.node.state[i][j]].append(tuple([i,j]))
                    else:
                        self.start_finish_dict[self.node.state[i][j]]=[tuple([i,j])]

        for key in self.start_finish_dict:
            self.sf_list.extend(self.start_finish_dict[key])
        return self.node

    def get_moves(self, node):
        
        i,j = node.current_pos
     
        working_val = node.state[i][j]
        possible_moves = []

        if j>0:
            possible_moves.append("left")
        if j<len(node.state)-1:
            possible_moves.append("right")
        if i >0:
            possible_moves.append("up")
        if i<len(node.state[i])-1:
            possible_moves.append("down")
            
        return possible_moves

 
    def execute_move(self, passed_node, move):
        
        new_node_state = self.copy(passed_node.state)
        
        i,j = passed_node.current_pos
        invalid=False
        
        match move:
            case "up":
                if HeuristicsV2.goal_state(move,passed_node,self):
                    self.start_finish_dict.pop(new_node_state[i][j])
                    self.closed_list.append(new_node_state[i][j])
                    val,pos = self.start_finish_dict.popitem()
                    current_pos=pos[0]
                    self.start_finish_dict[val]=pos
                elif HeuristicsV2.intersects_line(move,passed_node,self):
                    invalid=True
                else:
                    new_node_state[i-1][j] = passed_node.state[i][j]
                    current_pos=tuple([i-1,j])
                
            case "down":
                if HeuristicsV2.goal_state(move,passed_node,self):
                    print("landed in goal")
                    self.start_finish_dict.pop(new_node_state[i][j])
                    self.closed_list.append(new_node_state[i][j])
                    val,pos = self.start_finish_dict.popitem()
                    current_pos=pos[0]
                    
                    self.start_finish_dict[val]=pos
                elif HeuristicsV2.intersects_line(move,passed_node,self):
                    invalid=True
                else:
                    new_node_state[i+1][j] = passed_node.state[i][j]
                    current_pos=tuple([i+1,j])
        
            case "left":
                if HeuristicsV2.goal_state(move,passed_node,self):
                    print("how")
                    self.start_finish_dict.pop(new_node_state[i][j])
                    self.closed_list.append(new_node_state[i][j])
                    val,pos = self.start_finish_dict.popitem()
                    current_pos=pos[0]
                    print(val,current_pos)
                    self.start_finish_dict[val]=pos
                elif HeuristicsV2.intersects_line(move,passed_node,self):
                    invalid=True
                else:
                    new_node_state[i][j-1] = passed_node.state[i][j]
                    current_pos=tuple([i,j-1])
                    
            case "right":
                if HeuristicsV2.goal_state(move,passed_node,self):
                    self.start_finish_dict.pop(new_node_state[i][j])
                    self.closed_list.append(new_node_state[i][j])
                    val,pos = self.start_finish_dict.popitem()
                    current_pos=pos[0]
                    print(current_pos)
                    self.start_finish_dict[val]=pos
                elif HeuristicsV2.intersects_line(move,passed_node,self):
                    invalid=True
                else:
                    new_node_state[i][j+1] = passed_node.state[i][j]
                    current_pos=tuple([i,j+1])
                
        if invalid:
            invalid_node = node(new_node_state,tuple([-1,-1]),passed_node)
            invalid_node.heuristic=-1
            return invalid_node
                    
        return node(new_node_state,current_pos,passed_node)
        
    
    def sucessor_state_gen(self,passed_node):
        successor_states = []
        moves = self.get_moves(passed_node)
        print(f"Starting at position {passed_node.current_pos}")
        for move in moves:
            successor_states.append(self.execute_move(passed_node,move))
       
        for nodes in successor_states:
            state=nodes.state
            for row in state:
                print(row)
            print()
        
    def copy(self, state):
        return [row[:] for row in state]
    
class node:
    
    def __init__(self, state,current_pos,parent):
        self.state = state
        self.current_pos= current_pos
        self.heuristic=0
        self.parent=parent
        

In [471]:
class HeuristicsV2:

    @staticmethod
    def inadmissible_heuristics(current_node, move,puzzle):
        current_state = current_node.state
        
        if current_node.heuristic==-1:
            return current_node
        
        
        successor_node = puzzle.execute_move(current_node, move)
        ci,cj=successor_node.current_pos
        print(move,successor_node.state,[ci,cj])
        si,sj = puzzle.start_finish_dict.get(successor_node.state[ci][cj])[1]

        distance = abs(ci-si)+abs(cj-sj)

#         if distance!=1:
#             distance+= extra_distance(successor_node.state,ci,cj)
        successor_node.heuristics=distance

        return successor_node
    @staticmethod
    def admissible_heuristics(current_node, move):
        current_state = current_node.state
        
        
#         if HeuristicsV2.intersects_line(self,move,current_state,puzzle):
#                 shouldnt append to queue because its technically invalid we can assign it a value of 1M
#                  return tuple([10000000,puzzle.execute_move(current_node,move)])

        
# Guarantees only the correct number will be able to link itself with its start or stop state
    def intersects_line(move,current_node,puzzle):
        
        i,j=current_node.current_pos
        working_val = current_node.state[i][j]
        
        if move == "down":
            if (current_node.state[i+1][j]!=''):
                return True
        elif move == "up":
            if (current_node.state[i-1][j]!=''):
                return True
        elif move == "left":
            if (current_node.state[i][j-1]!=''):
                return True
        elif move == "right":
            if (current_node.state[i][j+1]!=''):
                return True
        return False

    @staticmethod
    def extra_distance(new_node_state,ci,cj):
        distance=0
        if new_node_state[ci-1][cj] != ' ':
            distance+1
        if new_node_state[ci+1][cj] != ' ': 
            distance+1
        if new_node_state[ci][cj+1] != ' ':
            distance+1
        if new_node_state[ci][cj-1] != ' ':
            distance+1
        return distance

    @staticmethod
    def goal_state(move,current_node,puzzle):
        i,j=current_node.current_pos
        working_val = current_node.state[i][j]
        if move == "down":
            if puzzle.start_finish_dict.get(working_val) !=None and tuple([i+1,j]) in puzzle.start_finish_dict.get(working_val):
                return True
        elif move == "up":
            return puzzle.start_finish_dict.get(working_val)!=None and tuple([i-1,j]) in puzzle.start_finish_dict.get(working_val)

        elif move == "left":
            return puzzle.start_finish_dict.get(working_val)!=None and tuple([i,j-1]) in puzzle.start_finish_dict.get(working_val)
        elif move == "right":
            return puzzle.start_finish_dict.get(working_val)!=None and tuple([i,j+1]) in puzzle.start_finish_dict.get(working_val)

    

In [472]:
def best_first(puzzle, heuristics):
    i = 0
    open_list = []
    closed_list = []
    start_node = puzzle.node
    open_list = [start_node]

    while open_list:
        open_list = sorted(open_list, key=lambda node: node.heuristic)
        current_node = open_list.pop(0)
        
#         Check if puzzle is finished 
        if len(puzzle.closed_list)== 4:
            return current_node.state,i
        
        else:
            for moves in puzzle.get_moves(current_node):
                
                match heuristics:
                    case "inadmissible_heuristics":
                        successor = HeuristicsV2.inadmissible_heuristics(current_node,moves,puzzle)
                        if successor.heuristic==-1:
                            continue
                    case "admissible_heuristics":
                        pass
                
                if successor not in closed_list and successor not in open_list:
                    open_list.append(successor)
            closed_list.append(successor)
            i=i+1
    return None


def a_star(puzzle, heuristics):
    closed_list = []
    start_state = puzzle.initial_state
    open_list = [tuple([1, start_state, 0])]
    i=0
    while open_list:
        open_list = sorted(open_list, key=lambda tup: tup[0])
        current_state_t = open_list.pop(0)

        current_state = current_state_t[1]
        depth = current_state_t[2]

        if current_state == puzzle.goal_state:
            puzzle.closed_list = closed_list
            return current_state_t,i


        else:
            for moves in puzzle.get_moves(current_state):
                state = puzzle.execute_move(current_state, moves)
                match heuristics:
                    case "hamming_distance":
                        successor_state = Heuristics.hamming_distance(puzzle, state)
                    case "manhattan_distance":
                        successor_state = Heuristics.manhattan_distance(puzzle, state)
                    case "permutation_distance":
                        successor_state = Heuristics.permutation_distance(puzzle, state)
                    case "inversion_manhattan":
                        successor_state = Heuristics.inversion_manhattan(puzzle, state)
                    case _:
                        successor_state = Heuristics.hamming_distance(puzzle, current_state)

                successor_state = tuple([(successor_state[0] + depth+ 1), successor_state[1], depth + 1])
#                 if state not in closed_list and state not in open_list:
#                     open_list.append(state)
#             closed_list.append(current_state_t)

                if not closed_list:
                    open_list.append(successor_state)

                #     remove closed states that have a higher cost
                execute = True
                to_remove = puzzle.copy(closed_list)
                for closed in to_remove:
                    if closed[1] == successor_state[1] and closed[0] > successor_state[0]:
                        open_list.append(successor_state)
                        closed_list.remove(closed)

                temp_open_list = puzzle.copy(open_list)
                for expanded in temp_open_list:
                    if expanded[1] == successor_state[1] and expanded[0] > successor_state[0]:
                        open_list.remove(expanded)
                        open_list.append(successor_state)
                        execute = False
                        break
                if execute:
                    open_list.append(successor_state)
        closed_list.append(current_state_t)
        i=i+1
    return None 
    

In [473]:
initial_state=[[3,'','',2],
               ['',2,1,''],
               ['','',3,1],
               [4,'','',4]]

In [474]:
numlink= numlink(initial_state)
state = numlink.declare_SF()

In [475]:
best_first(numlink,"inadmissible_heuristics")

left [[3, '', '', 2], ['', 2, 1, ''], ['', '', 3, 1], [4, '', '', 4]] [-1, -1]
right [[3, '', '', 2], ['', 2, 1, 1], ['', '', 3, 1], [4, '', '', 4]] [1, 3]
up [[3, '', 1, 2], ['', 2, 1, ''], ['', '', 3, 1], [4, '', '', 4]] [0, 2]
down [[3, '', '', 2], ['', 2, 1, ''], ['', '', 3, 1], [4, '', '', 4]] [-1, -1]
how
4 (3, 0)
left [[3, '', '', 2], ['', 2, 1, 1], ['', '', 3, 1], [4, '', '', 4]] [3, 0]
up [[3, '', '', 2], ['', 2, 1, 1], ['', '', 3, 1], [4, '', '', 4]] [-1, -1]
down [[3, '', '', 2], ['', 2, 1, 1], ['', '', 3, 1], [4, '', '', 4]] [-1, -1]
left [[3, 1, 1, 2], ['', 2, 1, ''], ['', '', 3, 1], [4, '', '', 4]] [0, 1]


TypeError: 'NoneType' object is not subscriptable